In [29]:
import os
import io
import numpy as np
from PIL import Image
import fitz
import re
import torch
import time
import datetime

from faster_rcnn import BradOCR, plot_text_xyxy, FasterRCNN, pre_process_image_bboxes, to_tensor, nms
from ocr_post_process import group_text_into_line, find_single_passage

- faster RCNN

In [ ]:
filepath_model_char=os.path.join(r'C:\Users\bomso\bomsoo1\python\_pytorch\pjt_faster_rcnn\trained\char\fasterrcnn_01270246_0.9668653988020294')
reloaded_char = torch.load(filepath_model_char, map_location=torch.device('cpu'))
n_test_pre_nms=12000
n_test_post_nms=2000
reloaded_char['model_init_inputs']['n_test_pre_nms'] = n_test_pre_nms # modify model inputs
reloaded_char['model_init_inputs']['n_test_post_nms'] = n_test_post_nms # modify model inputs
print(f">> reloaded_char['model_init_inputs'] = {reloaded_char['model_init_inputs']}")

faster_rcnn = FasterRCNN(**{
    'n_fg_class': 93, 'feat_stride': 16, 
    'loc_normalize_mean': (0.0, 0.0, 0.0, 0.0), 'loc_normalize_std': (0.1, 0.1, 0.2, 0.2), 'backbone_name': 'vgg16', 
    'use_pretrained_weights': True, 'freeze_param_up_to': 0, 'ceil_mode_for_MaxPool2d': True, 'always_freeze_first_batch_norm': False, 'use_untrained_vgg_classifier': False, 
    'base_size': 16, 
    'ratios': [0.25, 0.5, 1, 2, 4], 'anchor_scales': [0.25, 0.5, 1, 2, 4], 
    'y_offsets': [-5.333333333333333, 0, 5.333333333333333], 'x_offsets': [-5.333333333333333, 0, 5.333333333333333], 
    'num_offsets': None, 'nms_thresh': 0.7, 
    'n_train_pre_nms': 12000, 'n_train_post_nms': 2000, 
    # 'n_test_pre_nms': 12000, 'n_test_post_nms': 2000, # !!!!!!!!!!!!!!!!!! TIME-CONSUMPING
    'min_size': 0, 'roi_size': 7, 
    'is_roi_align': True, 'roi_align_sampling_ratio': 2, 'roi_align_aligned': True, 
    'atc_n_sample': 256, 'atc_pos_iou_thresh': 0.7, 'atc_neg_iou_thresh': 0.3, 'atc_pos_ratio': 0.5, 'atc_use_original_subsample_for_postive_labels': True, 
    'ptc_n_sample': 128, 'ptc_pos_ratio': 0.25, 'ptc_pos_iou_thresh': 0.5, 'ptc_neg_iou_thresh_hi': 0.5, 'ptc_neg_iou_thresh_lo': 0.0,
    })
# faster_rcnn = FasterRCNN(**reloaded_char['model_init_inputs'])
faster_rcnn.eval()
tic = time.time()
for i in range(3):
    imgs = torch.zeros((1,128,128*10,3))
    out = faster_rcnn(imgs)
    # img_text = np.zeros((100,1000,3))
    # imgs, _ = pre_process_image_bboxes(img_text, bboxes=None, fix_img_H=100)
    # pred_bboxes_char, pred_labels_char, pred_scores_char = faster_rcnn.predict(to_tensor(imgs[None]), visualize=True, visualize_score_thresh=0.05)

    # #--- nms ---------------------------------------------------------------------
    # ikeep = nms(torch.from_numpy(pred_bboxes_char[0]), torch.from_numpy(pred_scores_char[0]), 0.25)
    # pred_bbox_, pred_label_, pred_score_ = pred_bboxes_char[0][ikeep], pred_labels_char[0][ikeep], pred_scores_char[0][ikeep]
    # ii = pred_score_ > 0.4
    # pred_bbox, pred_label, pred_score = pred_bbox_[ii], pred_label_[ii], pred_score_[ii]
    # isort = np.argsort(pred_bbox[:,1] + pred_bbox[:,3])
    # pred_bbox, pred_label, pred_score = pred_bbox[isort], pred_label[isort], pred_score[isort]

    toc = time.time()
    print(toc - tic, '[sec]')
    tic = time.time()


>> reloaded_char['model_init_inputs'] = {'n_fg_class': 93, 'feat_stride': 16, 'loc_normalize_mean': (0.0, 0.0, 0.0, 0.0), 'loc_normalize_std': (0.1, 0.1, 0.2, 0.2), 'backbone_name': 'vgg16', 'use_pretrained_weights': True, 'freeze_param_up_to': 0, 'ceil_mode_for_MaxPool2d': True, 'always_freeze_first_batch_norm': False, 'use_untrained_vgg_classifier': False, 'base_size': 16, 'ratios': [0.25, 0.5, 1, 2, 4], 'anchor_scales': [0.25, 0.5, 1, 2, 4], 'y_offsets': [-5.333333333333333, 0, 5.333333333333333], 'x_offsets': [-5.333333333333333, 0, 5.333333333333333], 'num_offsets': None, 'nms_thresh': 0.7, 'n_train_pre_nms': 12000, 'n_train_post_nms': 2000, 'n_test_pre_nms': 12000, 'n_test_post_nms': 2000, 'min_size': 0, 'roi_size': 7, 'is_roi_align': True, 'roi_align_sampling_ratio': 2, 'roi_align_aligned': True, 'atc_n_sample': 256, 'atc_pos_iou_thresh': 0.7, 'atc_neg_iou_thresh': 0.3, 'atc_pos_ratio': 0.5, 'atc_use_original_subsample_for_postive_labels': True, 'ptc_n_sample': 128, 'ptc_pos_rat